# Refinement Procedure
for each dataset, do the following refinement process. it doesnt have to be executed in listed order.
1. Process miscellaneous things.
2. unit conversion
3. outlier deletion

In [1]:
import pandas as pd
from warnings import simplefilter
from tqdm import tqdm
import numpy as np
import datetime
import sys
simplefilter(action="ignore", category=pd.errors.DtypeWarning)
pd.set_option('display.max_columns', None)

## 1. EMAR

### 1.1 cohort selection

In [2]:
icustays = pd.read_csv('processed_data/sepsis/icustays_sepshock.csv')
icustays = icustays.loc[icustays.cohort_stays == 1].reset_index(drop=True)
hadmids = icustays.hadm_id.unique()
icustays.intime = pd.to_datetime(icustays.intime)
icustays.outtime = pd.to_datetime(icustays.outtime)

admissions = pd.read_csv('hosp/admissions.csv')
admissions.admittime = pd.to_datetime(admissions.admittime)
admissions.dischtime = pd.to_datetime(admissions.dischtime)

ie_bfcc = pd.read_csv('processed_data/sepsis/IE_BFCC.csv')
ie_bfcc.starttime = pd.to_datetime(ie_bfcc.starttime)
ie_bfcc.endtime = pd.to_datetime(ie_bfcc.endtime)

In [4]:
emar = pd.read_csv('hosp/emar.csv')
emar_detail = pd.read_csv('hosp/emar_detail.csv')

In [5]:
vaso_names = pd.read_csv('processed_data/sepsis/vasofluid_names.csv').vasofluid_names.tolist()[:6]
fluid_names = pd.read_csv('processed_data/sepsis/vasofluid_names.csv').vasofluid_names.tolist()[6:]
a_keys_nl = ['DOP', 'PHENYLEPH', 'NOR', 'EPINEPH', 'DOBUT', 'EpiPEN']
a_keys_l = ['adrenaline', 'crystal']
d_keys = ['oral', 'crrt', 'racepinephrine', 'lidocaine', 'chewable', 'orthopedic solution', 'fluocinolone acetonide', 'clindamycin', 'ophth. soln']

In [43]:
# first condition: data from cohorts
tmp = emar.medication.astype('str')
# tmp_cond = (tmp.str.lower().str.contains('|'.join(vaso_names+fluid_names+a_keys_l)) | tmp.str.contains('|'.join(a_keys_nl))) & (~tmp.str.lower().str.contains('|'.join(d_keys))) & (emar.hadm_id.isin(hadmids))
tmp_cond = (emar.hadm_id.isin(hadmids))
tmp_emar = emar.loc[tmp_cond].reset_index(drop=True)
print(tmp_emar.medication.unique())
print(tmp_emar.info())
print(f'# of admissions: {len(tmp_emar.hadm_id.unique())}')
tmp_emar.charttime = pd.to_datetime(tmp_emar.charttime)# for second condtion application

['Albuterol 0.083% Neb Soln' 'Ipratropium Bromide Neb' nan ...
 'Midodrine 20 mg TID or Placebo' 'benazepril'
 'benazepril-hydrochlorothiazide']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3556133 entries, 0 to 3556132
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   subject_id    int64  
 1   hadm_id       float64
 2   emar_id       object 
 3   emar_seq      int64  
 4   poe_id        object 
 5   pharmacy_id   float64
 6   charttime     object 
 7   medication    object 
 8   event_txt     object 
 9   scheduletime  object 
 10  storetime     object 
dtypes: float64(2), int64(2), object(7)
memory usage: 298.4+ MB
None
# of admissions: 4726


In [44]:
# second condition: st, et 이전 이후 데이터 이면서 at-3d, dt+3d 사이 데이터일 것.
including_idx = []
td = datetime.timedelta(days=3)
for adm in tqdm(tmp_emar.hadm_id.unique()):
    tmp_cond = admissions.hadm_id == adm
    at, dt = admissions.loc[tmp_cond, ('admittime', 'dischtime')].reset_index(drop=True).loc[0]

    tmp_cond = (tmp_emar.hadm_id == adm) & (tmp_emar.charttime >= (at-td)) & (tmp_emar.charttime <= (dt+td))
    tmp_df_part = tmp_emar.loc[tmp_cond]
    
    tmp_cond = icustays.hadm_id == adm
    tmp_stayids = icustays.loc[tmp_cond].stay_id.tolist()
    cond = pd.Series([True for _ in range(len(tmp_df_part))], index=tmp_df_part.index)
    for stay in tmp_stayids:
        if len(ie_bfcc.loc[ie_bfcc.stay_id == stay]) == 0:
            continue
        st = min(ie_bfcc.loc[ie_bfcc.stay_id == stay].starttime)
        et = max(ie_bfcc.loc[ie_bfcc.stay_id == stay].endtime)
        tmp_cond = (tmp_df_part.charttime < st) | (tmp_df_part.charttime > et)
        cond = cond & tmp_cond
    
    including_idx += cond[cond].index.tolist()
including_idx = list(set(including_idx))
tmp_emar.loc[including_idx].reset_index(drop=True)

100%|██████████| 4726/4726 [02:45<00:00, 28.57it/s]


,subject_id,hadm_id,emar_id,emar_seq,poe_id,pharmacy_id,charttime,medication,event_txt,scheduletime,storetime
0,10001884,26184834.0,10001884-1016,1016,10001884-1525,56851269.0,2131-01-08 00:30:00,Albuterol 0.083% Neb Soln,Administered,2131-01-08 00:52:00,2131-01-08 00:52:00
1,10001884,26184834.0,10001884-1017,1017,10001884-1524,1448699.0,2131-01-08 00:30:00,Ipratropium Bromide Neb,Administered,2131-01-08 01:00:00,2131-01-08 00:52:00
2,10001884,26184834.0,10001884-1018,1018,10001884-1529,20424308.0,2131-01-08 00:52:00,NaN,Administered,2131-01-08 01:00:00,2131-01-08 00:53:00
3,10001884,26184834.0,10001884-1019,1019,10001884-1506,79472347.0,2131-01-08 01:21:00,Docusate Sodium,Administered,2131-01-08 00:38:00,2131-01-08 01:21:00
4,10001884,26184834.0,10001884-1020,1020,10001884-1505,48536144.0,2131-01-08 01:21:00,Atorvastatin,Administered,2131-01-08 00:38:00,2131-01-08 01:21:00
...,...,...,...,...,...,...,...,...,...,...,...
1659728,19995595,21784060.0,19995595-995,995,19995595-762,6085795.0,2126-11-06 16:04:00,NaN,Stopped,2126-11-06 16:04:00,2126-11-06 16:04:00
1659729,19995595,21784060.0,19995595-996,996,19995595-782,63537437.0,2126-11-06 16:04:00,HydrALAZINE,Administered,2126-11-06 16:04:00,2126-11-06 16:04:00
1659730,19995595,21784060.0,19995595-997,997,19995595-785,NaN,2126-11-06 17:42:00,NIFEdipine (Extended Release),Not Given,2126-11-06 17:42:00,2126-11-06 17:48:00
1659731,19995595,21784060.0,19995595-998,998,19995595-786,49659760.0,2126-11-06 17:50:00,Captopril,Administered,2126-11-06 17:24:00,2126-11-06 17:51:00


In [45]:
pharmacy = pd.read_csv('hosp/pharmacy.csv')

In [49]:
tmp_cond = pharmacy.pharmacy_id == 88825510.0
pharmacy.loc[tmp_cond]

,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,verifiedtime,route,frequency,disp_sched,infusion_type,sliding_scale,lockout_interval,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
14277569,19990427,29695607,88825510,19990427-104,2182-01-05 09:00:00,2182-01-14 04:00:00,NaN,IV Large Volume,Discontinued,2182-01-05 08:49:33,2182-01-05 08:49:33,IV,ASDIR,NaN,C,N,NaN,NaN,NaN,NaN,NaN,Ongoing,NaN,Enter on Label,NaN,Distribution-Floor Stock,NaN


In [50]:
tmp_cond = poe.poe_id == '19990427-104'
poe.loc[tmp_cond]

,poe_id,poe_seq,subject_id,hadm_id,ordertime,order_type,order_subtype,transaction_type,discontinue_of_poe_id,discontinued_by_poe_id,order_status
41385911,19990427-104,104,19990427,29695607,2182-01-06 09:35:41,IV therapy,IV fluids,New,NaN,NaN,Inactive


In [51]:
tmp_cond = emar.poe_id == '19990427-104'
emar.loc[tmp_cond]

,subject_id,hadm_id,emar_id,emar_seq,poe_id,pharmacy_id,charttime,medication,event_txt,scheduletime,storetime
28163840,19990427,29695607.0,19990427-111,111,19990427-104,88825510.0,2182-01-07 17:24:00,NaN,Confirmed,2182-01-07 17:24:00,2182-01-07 17:24:00
28163850,19990427,29695607.0,19990427-120,120,19990427-104,88825510.0,2182-01-07 23:59:00,NaN,Confirmed,2182-01-07 23:59:00,2182-01-07 23:59:00
28163863,19990427,29695607.0,19990427-132,132,19990427-104,88825510.0,2182-01-08 09:21:00,NaN,Confirmed,2182-01-08 09:21:00,2182-01-08 09:21:00
28163869,19990427,29695607.0,19990427-138,138,19990427-104,88825510.0,2182-01-08 10:35:00,NaN,Confirmed,2182-01-08 10:35:00,2182-01-08 10:35:00
28163877,19990427,29695607.0,19990427-145,145,19990427-104,88825510.0,2182-01-08 20:03:00,NaN,Confirmed,2182-01-08 20:03:00,2182-01-08 20:03:00
28163888,19990427,29695607.0,19990427-155,155,19990427-104,88825510.0,2182-01-09 02:04:00,NaN,Confirmed,2182-01-09 02:04:00,2182-01-09 02:04:00
28163889,19990427,29695607.0,19990427-156,156,19990427-104,88825510.0,2182-01-09 05:30:00,NaN,Confirmed,2182-01-09 05:30:00,2182-01-09 05:30:00
28163909,19990427,29695607.0,19990427-174,174,19990427-104,88825510.0,2182-01-09 23:11:00,NaN,Confirmed,2182-01-09 23:11:00,2182-01-09 23:11:00
28163919,19990427,29695607.0,19990427-183,183,19990427-104,88825510.0,2182-01-10 06:01:00,NaN,Confirmed,2182-01-10 06:01:00,2182-01-10 06:04:00
28163928,19990427,29695607.0,19990427-191,191,19990427-104,88825510.0,2182-01-10 09:20:00,NaN,Confirmed,2182-01-10 09:20:00,2182-01-10 09:20:00


In [12]:
tmp_emar = tmp_emar.loc[including_idx].reset_index(drop=True)

In [16]:
tmp = tmp_emar.medication.astype('str')
tmp_cond = (tmp.str.lower().str.contains('|'.join(vaso_names)) | tmp.str.contains('|'.join(a_keys_nl)))
print(f'# of rows for vaso: {sum(tmp_cond)}')
print(f'# of admissions for vaso: {len(tmp_emar.loc[tmp_cond].hadm_id.unique())}')

# of rows for vaso: 3725
# of rows for vaso: 676


In [17]:
tmp_cond = emar_detail.emar_id.isin(tmp_emar.emar_id.unique().tolist())
tmp_emdt = emar_detail.loc[tmp_cond].reset_index(drop=True)

In [42]:
emar.loc[(emar.subject_id == 19990427) & (emar.emar_seq >= 80) & (emar.emar_seq <= 90)]

,subject_id,hadm_id,emar_id,emar_seq,poe_id,pharmacy_id,charttime,medication,event_txt,scheduletime,storetime
28164588,19990427,29695607.0,19990427-80,80,19990427-104,88825510.0,2182-01-06 20:28:00,NaN,Confirmed,2182-01-06 20:28:00,2182-01-06 20:29:00
28164589,19990427,29695607.0,19990427-81,81,19990427-94,NaN,2182-01-06 20:00:00,Sodium Chloride 0.9% Flush,Not Flushed,2182-01-06 20:00:00,2182-01-06 20:30:00
28164590,19990427,29695607.0,19990427-82,82,19990427-109,NaN,2182-01-06 20:00:00,Sodium Chloride 0.9% Flush,Not Flushed,2182-01-06 20:00:00,2182-01-06 20:30:00
28164591,19990427,29695607.0,19990427-83,83,19990427-135,NaN,2182-01-06 17:15:00,PHENYLEPHrine,Stopped in Other Location,2182-01-06 17:15:00,2182-01-06 20:53:00
28164592,19990427,29695607.0,19990427-84,84,19990427-130,NaN,2182-01-06 14:38:00,Bupivacaine 0.1%,Not Started,2182-01-06 14:38:00,2182-01-06 20:54:00
28164593,19990427,29695607.0,19990427-85,85,19990427-98,78442606.0,2182-01-06 23:59:00,HYDROmorphone (Dilaudid),Assessed,2182-01-06 23:59:00,2182-01-07 00:02:00
28164594,19990427,29695607.0,19990427-86,86,19990427-99,NaN,2182-01-07 00:00:00,Sodium Chloride 0.9% Flush,Not Flushed,2182-01-07 00:00:00,2182-01-07 00:03:00
28164595,19990427,29695607.0,19990427-87,87,19990427-120,90353606.0,2182-01-07 02:39:00,CeFAZolin,Administered,2182-01-07 02:00:00,2182-01-07 02:40:00
28164596,19990427,29695607.0,19990427-88,88,19990427-98,78442606.0,2182-01-07 02:39:00,HYDROmorphone (Dilaudid),Assessed,2182-01-07 02:39:00,2182-01-07 02:40:00
28164597,19990427,29695607.0,19990427-89,89,19990427-104,88825510.0,2182-01-07 02:39:00,NaN,Confirmed,2182-01-07 02:39:00,2182-01-07 02:40:00


In [33]:
emar_detail.loc[emar_detail.emar_id == '19990427-83']

,subject_id,emar_id,emar_seq,parent_field_ordinal,administration_type,pharmacy_id,barcode_type,reason_for_no_barcode,complete_dose_not_given,dose_due,dose_due_unit,dose_given,dose_given_unit,will_remainder_of_dose_be_given,product_amount_given,product_unit,product_code,product_description,product_description_other,prior_infusion_rate,infusion_rate,infusion_rate_adjustment,infusion_rate_adjustment_amount,infusion_rate_unit,route,infusion_complete,completion_interval,new_iv_bag_hung,continued_infusion_in_other_location,restart_interval,side,site,non_formulary_visual_verification
57418632,19990427,19990427-83,83,NaN,IV Infusion,NaN,NaN,NaN,NaN,NaN,mcg/kg/min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mcg/kg/min,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
poe = pd.read_csv('hosp/poe.csv')
poe_detail = pd.read_csv('hosp/poe_detail.csv')

In [52]:
poe.loc[poe.subject_id == 19990427].sort_values('poe_seq')

,poe_id,poe_seq,subject_id,hadm_id,ordertime,order_type,order_subtype,transaction_type,discontinue_of_poe_id,discontinued_by_poe_id,order_status
41386439,19990427-6,6,19990427,29695607,2182-01-04 09:07:06,ADT orders,Admit,New,NaN,NaN,Inactive
41386462,19990427-7,7,19990427,29695607,2182-01-04 09:22:16,Blood Bank,Blood tests,New,NaN,NaN,Inactive
41385927,19990427-12,12,19990427,29695607,2182-01-04 09:42:28,ADT orders,Postop,New,NaN,NaN,Inactive
41385937,19990427-13,13,19990427,29695607,2182-01-04 09:42:28,General Care,Vitals/Monitoring,New,NaN,NaN,Inactive
41385947,19990427-14,14,19990427,29695607,2182-01-04 09:42:28,General Care,Activity,New,NaN,NaN,Inactive
...,...,...,...,...,...,...,...,...,...,...,...
41386448,19990427-607,607,19990427,29695607,2182-01-24 09:14:29,Medications,NaN,New,NaN,19990427-610,Inactive
41386449,19990427-608,608,19990427,29695607,2182-01-24 09:14:29,Medications,NaN,D/C,19990427-588,NaN,Inactive
41386450,19990427-609,609,19990427,29695607,2182-01-24 10:49:32,Medications,NaN,New,NaN,NaN,Inactive
41386452,19990427-610,610,19990427,29695607,2182-01-24 16:07:32,Medications,NaN,Change,19990427-607,19990427-611,Inactive


In [39]:
poe_detail.loc[poe_detail.subject_id == 19990427]

,poe_id,poe_seq,subject_id,field_name,field_value
3171824,19990427-106,106,19990427,Tubes & Drains type,Indwelling urinary catheter (IUC) - Foley
3171825,19990427-112,112,19990427,Tubes & Drains type,Closed suction drain
3171826,19990427-12,12,19990427,Admit category,Admit to inpatient
3171827,19990427-155,155,19990427,Tubes & Drains type,Intermittent straight cath
3171828,19990427-161,161,19990427,Tubes & Drains type,Indwelling urinary catheter (IUC) - Foley
3171829,19990427-163,163,19990427,Tubes & Drains type,Indwelling urinary catheter (IUC) - Foley
3171830,19990427-168,168,19990427,Tubes & Drains type,Indwelling urinary catheter (IUC) - Foley
3171831,19990427-185,185,19990427,Tubes & Drains type,Indwelling urinary catheter (IUC) - Foley
3171832,19990427-193,193,19990427,Tubes & Drains type,Indwelling urinary catheter (IUC) - Foley
3171833,19990427-211,211,19990427,Admit category,Admit to inpatient


# EON